# Introduction 

Term deposits are a major source of income for a bank. A term deposit is a cash investment held at a financial institution. Your money is invested for an agreed rate of interest over a fixed amount of time, or term. The bank has various outreach plans to sell term deposits to their customers such as email marketing, advertisements, telephonic marketing, and digital marketing.

Telephonic marketing campaigns still remain one of the most effective way to reach out to people. However, they require huge investment as large call centers are hired to actually execute these campaigns. Hence, it is crucial to identify the customers most likely to convert beforehand so that they can be specifically targeted via call. 

In this notebook I will create a profile for the users more likely to perform a term deposit.

## Columns

1. age (numeric)
2. job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
"blue-collar","self-employed","retired","technician","services")
3. marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
4. education (categorical: "unknown","secondary","primary","tertiary")
5. default: has credit in default? (binary: "yes","no")
6. balance: average yearly balance, in euros (numeric)
7. housing: has housing loan? (binary: "yes","no")
8. loan: has personal loan? (binary: "yes","no")
9. contact: contact communication type (categorical: "unknown","telephone","cellular")
10. day: last contact day of the month (numeric)
11. month: last contact month of year (categorical: "jan", "feb", "mar", …, "nov", "dec")
12. duration: last contact duration, in seconds (numeric)
13. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
14. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client wasnot previously contacted)
15. previous: number of contacts performed before this campaign and for this client (numeric)
16. poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")
17. y - has the client subscribed a term deposit? (binary: "yes","no")

In [94]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from functools import reduce

In [74]:
df = pd.read_csv("data/data.csv", sep=";")
df["y_numeric"] = df["y"].map({"no": 0, "yes": 1})
assert not df.isna().any().any() # Assert that there are no missing values
df.shape

(45211, 18)

In [75]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,y_numeric
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,0


In [64]:
fig = px.bar(df["y"].value_counts(), title=f"Success Rate {int(round(5289/39922, 2)*100)}%") 
fig.layout.yaxis.title = "# Customers"
fig.layout.xaxis.title = "Successfull contact"
fig.layout.showlegend = False
fig.show()

In [105]:
def plot_monthly_success(df: pd.DataFrame):
    ordered_months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    plot_df = df[["month", "y_numeric"]].groupby("month").agg(
        n_calls=pd.NamedAgg(column="y_numeric", aggfunc=lambda x: x.shape[0]),
        success_rate=pd.NamedAgg(column="y_numeric", aggfunc=lambda x: (x.sum()/x.shape[0])*100)
        ).loc[ordered_months, :]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=plot_df.index, y=plot_df["n_calls"], name="# Calls"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=plot_df.index, y=plot_df["success_rate"], name="Success Rate"),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="Double Y Axis Example"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Month")

    # Set y-axes titles
    fig.update_yaxes(title_text="# Calls", secondary_y=False)
    fig.update_yaxes(title_text="Success Rate [%]", secondary_y=True)

    fig.show()
plot_monthly_success(df)

In [62]:
df[["job", "marital", "education", "default", "housing", "loan", "y_numeric"]].groupby(["job", "marital", "education", "default", "housing", "loan"], as_index=False)["y_numeric"].sum()

,job,marital,education,default,housing,loan,y_numeric
0,admin.,divorced,primary,no,no,no,0
1,admin.,divorced,primary,no,no,yes,0
2,admin.,divorced,primary,no,yes,no,3
3,admin.,divorced,primary,no,yes,yes,0
4,admin.,divorced,secondary,no,no,no,30
...,...,...,...,...,...,...,...
693,unknown,single,secondary,no,yes,no,0
694,unknown,single,tertiary,no,no,no,3
695,unknown,single,unknown,no,no,no,6
696,unknown,single,unknown,no,yes,no,0
